# Modelagem Parte 1: Árvore de Decisão e KNN 

**Responsável:** Clara

**Objetivo:** Nesta etapa, realizamos o treinamento dos algoritmos **Decision Tree** e **K-Nearest Neighbors (KNN)** utilizando o dataset pré-processado de saúde fetal.

**Metodologia:**
* **Validação:** 10-fold Stratified Cross-Validation.
* **Otimização:** `GridSearch` para encontrar os melhores hiperparâmetros (testando no mínimo 3 combinações).
* **Métricas:** Acurácia, Precisão, Recall e F1-Score (*weighted*).

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

import sys
import os

# Adiciona a raiz do projeto ao PYTHONPATH
sys.path.append(os.path.abspath(".."))


from src.modeling import run_classification_experiment

data_path = os.path.join('..', 'data', 'processed', 'fetal_health_processed.csv')
df = pd.read_csv(data_path)

X = df.drop('fetal_health', axis=1)
y = df['fetal_health']

print("Dados carregados com sucesso!")

Dados carregados com sucesso!


## 1. Árvore de Decisão (Decision Tree)

A árvore de decisão é um modelo interpretável que divide os dados com base em regras de decisão. Para evitar *overfitting* e encontrar a melhor generalização, testaremos os seguintes hiperparâmetros via `GridSearch`:

* **Criterion:** `gini` vs `entropy` (avalia qual métrica de pureza funciona melhor na divisão dos nós).
* **Max Depth:** `None` (sem limite), `10`, `20`, `30` (controla a profundidade máxima para limitar a complexidade da árvore).
* **Min Samples Split:** Controla o número mínimo de amostras necessárias para dividir um nó interno.

In [2]:
dt_model = DecisionTreeClassifier(random_state=42)

dt_params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 10, 20]
}

# Chama a função que está no arquivo .py
dt_grid = run_classification_experiment(dt_model, dt_params, X, y, 'DecisionTree')

print(f"Melhores parâmetros DT: {dt_grid.best_params_}")

--- Iniciando o GridSearch para DecisionTree ---
Fitting 10 folds for each of 24 candidates, totalling 240 fits
Melhor F1 para DecisionTree: 0.9340
Resultados salvos em: c:\Users\ga618\.vscode\Arquivos\InteligenciaArtificial\Projeto\MachineLearnig-Classifiacacao\results\metrics\DecisionTree_results.csv
Melhores parâmetros DT: {'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 20}


## 2. K-Nearest Neighbors (KNN)

O KNN classifica instâncias com base na proximidade com exemplos vizinhos. É sensível à escala dos dados (etapa já tratada no pré-processamento). Testaremos:

* **N Neighbors (k):** `3`, `5`, `7`, `9`, `11`, `15` (número de vizinhos considerados).
* **Weights:** `uniform` (todos têm peso igual) vs `distance` (vizinhos mais próximos têm maior influência na votação).
* **Metric:** `euclidean` (distância padrão) vs `manhattan` (soma das diferenças absolutas).

In [3]:
knn_model = KNeighborsClassifier()

knn_params = {
    'n_neighbors': [3, 5, 7, 9, 11, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

knn_grid = run_classification_experiment(knn_model, knn_params, X, y, 'KNN')

print(f"Melhores parâmetros KNN: {knn_grid.best_params_}")

--- Iniciando o GridSearch para KNN ---
Fitting 10 folds for each of 24 candidates, totalling 240 fits
Melhor F1 para KNN: 0.9077
Resultados salvos em: c:\Users\ga618\.vscode\Arquivos\InteligenciaArtificial\Projeto\MachineLearnig-Classifiacacao\results\metrics\KNN_results.csv
Melhores parâmetros KNN: {'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'distance'}


## 3. Naive Bayes

O algoritmo Naive Bayes é um classificador probabilístico fundamentado no Teorema de Bayes, assumindo uma forte independência "ingênua" entre os atributos. Para dados contínuos, como neste dataset, utilizamos a variante Gaussiana e testaremos o seguinte parâmetro:

* **Var Smoothing:** Parâmetro de estabilidade para cálculo de probabilidades (adiciona uma pequena variância para evitar erros de divisão por zero ou probabilidades nulas).


In [ ]:
from sklearn.naive_bayes import GaussianNB

nb_model = GaussianNB()

nb_params = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]
}

nb_grid = run_classification_experiment(nb_model, nb_params, X, y, 'NaiveBayes')

print(f"Melhores parâmetros NB: {nb_grid.best_params_}")

--- Iniciando o GridSearch para NaiveBayes ---
Fitting 10 folds for each of 4 candidates, totalling 40 fits
Melhor F1 para NaiveBayes: 0.8299
Resultados salvos em: c:\Users\ga618\.vscode\Arquivos\InteligenciaArtificial\Projeto\MachineLearnig-Classifiacacao\results\metrics\NaiveBayes_results.csv
Melhores parâmetros NB: {'var_smoothing': 1e-09}


## 4. Regressão Logística (Logistic Regression)

A Regressão Logística é usada para classificar dados calculando a probabilidade disso acontecer. Para melhorar o resultado, ajustamos a regularização simplificando o modelo:

* **C:** Inverso da força de regularização (valores menores indicam regularização mais forte, prevenindo *overfitting*).
* **Penalty:** 

`l1` (Lasso) :Pode zerar o peso de características inúteis.

`l2` (Ridge) Apenas reduz o peso das características menos importantes.

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(random_state=42, solver='liblinear')

lr_params = {
    # 'C' é o inverso da força de regularização. Valores menores significam regularização mais forte.
    'C': [0.1, 1.0, 10.0],
    # 'penalty' define o tipo de regularização: l1 ou l2. 
    'penalty': ['l1', 'l2']
}

lr_grid = run_classification_experiment(lr_model, lr_params, X, y, 'LogisticRegression')

print(f"Melhores parâmetros LR: {lr_grid.best_params_}")

--- Iniciando o GridSearch para LogisticRegression ---
Fitting 10 folds for each of 6 candidates, totalling 60 fits


c:\Users\ga618\.vscode\Arquivos\InteligenciaArtificial\Projeto\MachineLearnig-Classifiacacao\myfirstproject\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(


Melhor F1 para LogisticRegression: 0.8849
Resultados salvos em: c:\Users\ga618\.vscode\Arquivos\InteligenciaArtificial\Projeto\MachineLearnig-Classifiacacao\results\metrics\LogisticRegression_results.csv
Melhores parâmetros LR: {'C': 10.0, 'penalty': 'l1'}


## 5. Multi-layer Perceptron (MLP)

O MLP é uma rede neural artificial *feedforward* capaz de capturar relações não-lineares complexas nos dados através de múltiplas camadas de neurônios. O ajuste de sua arquitetura e regularização é essencial:

* **Hidden Layer Sizes:** Define a topologia da rede (quantidade de camadas ocultas e número de neurônios em cada uma).
* **Activation:** `tanh` vs `relu` (função de ativação não-linear aplicada aos neurônios das camadas ocultas).
* **Alpha:** Parâmetro de penalidade L2 para controlar a magnitude dos pesos e evitar *overfitting*.

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp_model = MLPClassifier(random_state=42, max_iter=500)

mlp_params = {
    'hidden_layer_sizes': [(50,), (50, 50), (100,)],

    'activation': ['tanh', 'relu'],
    
    'alpha': [0.0001, 0.001, 0.01]
}

mlp_grid = run_classification_experiment(mlp_model, mlp_params, X, y, 'MLP')

print(f"Melhores parâmetros MLP: {mlp_grid.best_params_}")

--- Iniciando o GridSearch para MLP ---
Fitting 10 folds for each of 18 candidates, totalling 180 fits
Melhor F1 para MLP: 0.8911
Resultados salvos em: c:\Users\ga618\.vscode\Arquivos\InteligenciaArtificial\Projeto\MachineLearnig-Classifiacacao\results\metrics\MLP_results.csv
Melhores parâmetros MLP: {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (100,)}
